## 0.0 파이썬 모듈 불러들이기

In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

## 0.1 하이퍼 파라미터 정의

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

## A.1 실험용 메인함수

In [3]:
def abalone_exec(epoch_count = 10, mb_size = 10, report = 1):
    load_abalone_dataset()  # 데이터 불러들이는 함수
    init_model()  # 모델 초기화 함수
    train_and_test(epoch_count, mb_size, report)   # 학습 및 테스트 수행 함수

## B.1 데이터 적재함수 정의 

In [4]:
def load_abalone_dataset():
    with open('./data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)   # 컬럼명을 건너 뛰어줌
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt  # cnt는 count
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for n, row in enumerate(rows):
        # 원-핫 벡터 처리
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]  # 성별을 지정해준 후 뒤의 나머지 정보를 붙혀줌

## B.2 파라미터 초기화 함수 정의 

In [5]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

## B.3 학습 및 평가 함수 정의

In [6]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) # 몇개의 미니배치를 만들지 
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

## C.1~3 학습 및 평가 데이터 획득 함수 정의

In [25]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

## C.4~5 학습 실행 함수와 평가 실행함수 정의

In [26]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

## D.1 / E.1 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [27]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

## D.2 / E.2 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [28]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

## D.3 정확도 계산 함수 정의  

In [29]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

## 단층퍼셉트론 메인 함수 실행

In [30]:
abalone_exec()

Epoch 1: loss=7.383, accuracy=0.803/0.831
Epoch 2: loss=6.241, accuracy=0.815/0.843
Epoch 3: loss=5.797, accuracy=0.822/0.838
Epoch 4: loss=5.561, accuracy=0.827/0.834
Epoch 5: loss=5.398, accuracy=0.829/0.849
Epoch 6: loss=5.339, accuracy=0.831/0.847
Epoch 7: loss=5.308, accuracy=0.831/0.837
Epoch 8: loss=5.218, accuracy=0.832/0.854
Epoch 9: loss=5.158, accuracy=0.832/0.841
Epoch 10: loss=5.258, accuracy=0.832/0.842

Final Test: final accuracy = 0.842


## 파라미터 확인

In [31]:
print(weight)
print(bias)

[[  0.58064234]
 [  1.05557514]
 [  1.32215129]
 [  4.37655336]
 [  5.36254398]
 [  4.53065651]
 [  5.31967816]
 [-16.26366599]
 [ -3.03748979]
 [ 12.0673827 ]]
[2.96651419]


## 새로운 입력 벡터 X에 대한 예측

In [32]:
x = np.array([0,1,0,0.44,0.3,0.08,0.5,0.23,0.11,0.2])
output = forward_neuralnet(x)
print(output)

(array([8.91753709]), array([0.  , 1.  , 0.  , 0.44, 0.3 , 0.08, 0.5 , 0.23, 0.11, 0.2 ]))


## 하이퍼퍼라미터 수정하며 실험

* LEARNING_RATE = 0.001 -> 0.1
* epoch_count = 10 -> 100
* mb_size = 10 -> 100

In [35]:
LEARNING_RATE = 0.1
abalone_exec(epoch_count=100,mb_size=100,report=20)

Epoch 20: loss=5.766, accuracy=0.826/0.826
Epoch 40: loss=5.255, accuracy=0.834/0.837
Epoch 60: loss=5.035, accuracy=0.839/0.833
Epoch 80: loss=4.953, accuracy=0.841/0.828
Epoch 100: loss=4.931, accuracy=0.841/0.836

Final Test: final accuracy = 0.836
